**TMDB API (Practice)** 

This practice assignment will reinforce important learning objectives from the previous lesson(s), and allow you to take on more challenging core assignments, preparing you for graduation.

Practice and tinker with this assignment until you're comfortable performing each of the tasks. Then, be sure to submit your output as described in the steps below.

**TMDB API (Practice):**

**Project Planning**

As discussed in the previous lesson, for the next part of your project, you will extract financial and certification data from TMDB's API for your IMDB data set. You will use an OUTER and INNER loop: a loop within a loop!

The OUTER loop will loop through the start years included in the IMDB data, filter the title basics data for the selected year, and save the list of movie ids from that year to retrieve in the inner loop.

The INNER loop loops through every movie id from the selected year, extracts its results from the TMDB API, and appends them to a JSON file.

# **For this practice assignment**

You will be practicing the inner loop of API calls for a single year's list of movies from your IMDB title basics data. Specifically, you will extract the API results for every movie with a startYear of 2000.

* **Read the instructions below, including the examples in the "Getting Started" section, before starting your work.**

* **Create a new notebook in your project repository called "Practicing TMDB API calls.**

**Preparation BEFORE the loop**
* Designate a folder to save your information.
* Define custom functions you will use for your API calls
* Load your cleaned title basics data from Part 1 of Project 2 (or query your title_basics table in your MySQL database).
* Define the year you wish to retrieve (2010) and create an empty list for appending error messages.

**Prepare the DataFrame and JSON File**
* **Use the selected year to define filenames and filter the data**
    1. Define a JSON_FILE filename to save the results in progress.
    2. Check if the file exists.
        * if it does not exist, create the empty JSON file with with open that just contains the key "imdb_id"
        * if it exists, print a message saying that it already exists.

***Now that the JSON file for the results in progress exists:***
* Filter the IMDB title basics data for the selected year and save the movie IDs from that year as "movies_ids".
* Check the JSON file for previously downloaded movie IDs and filter out the movie ids that already exists in the JSON file ( to prevent duplicate API calls) by:
    * Loading in the contents of the JSON file pd.read_json.
        * Compare the movie_ids that were in the JSON file to your saved movie_ids_to_get.
    * Save the final list of "movie_ids_to_get" by filtering out movies that already exists in the JSON file.

**Perform the Loop of API Calls**

Note: you have already written a function to combine the certification with the rest of the .info() from the TMDB API results in the Intro to TMDB API lesson.

**Create a loop to make API calls for each id** in the YEAR specified. Include a progress bar using tqdm_notebook.

***For each movie id:***
* Extract the current ID from the API and retrieve the dictionary of results
* Append the new results to the list from the JSON file
* Save the updated JSON file back to the disk

**Save the Results to Compressed .csv**
* **After the loop**, save the final results for the year as a csv.gz file with the year in the filename.

Note: at this point, you'll have completed the inner loop that you will need for the next part of the project.

# **Getting Started**

## **Preparation BEFORE the Loop:**

**Install, Import packages**

In [1]:
pip install tmdbsimple

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import packages
import os, time, json
import pandas as pd

In [3]:
from tqdm.notebook import tqdm_notebook

In [4]:
import tmdbsimple as tmdb

**Designate a folder**

You will save API call data in the data folder you created for project Part 1.

In [5]:
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title-akas-us-only.csv',
 'title-basics-us-only.csv',
 'title-ratings-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

In [6]:
# If you created the data folder for part 1, you will see your csv files listed here. If not, it will just be empty [].